In [2]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from matplotlib import pyplot as plt
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn import tree
import seaborn as sns

from sklearn.preprocessing import scale, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import sklearn

import xgboost as xgb
from xgboost import XGBClassifier

# Load the data 

In [5]:
def load(x,y,z,c,v): 
    df_x = pd.read_csv(x)
    df_y = pd.read_csv(y)
    df_z = pd.read_csv(z)
    df_c = pd.read_csv(c)
    df_v = pd.read_csv(v)
    return df_x, df_y, df_z, df_c, df_v


def cleanup(df_x, df_y, df_z, df_c, df_v):
    df_x = df_x[["pid","study_yr"]]
    df_y = df_y[["race","gender","age","scr_res0","scr_res1","scr_res2","pid"]]
    df_z = df_z[["sct_ab_desc","sct_ab_num","study_yr","pid"]]
    df_c = df_c.drop(columns=["visible_days","dataset_version"])
    df_v = df_v[["pid","de_stag","lesionsize","lc_morph","lc_behav","lc_grade","study_yr","lc_order"]]
    return df_x, df_y, df_z, df_c, df_v


def data(df_x, df_y, df_z, df_c, df_v):
    
    df_c[["sct_ab_attn","sct_ab_gwth"]] = df_c[["sct_ab_attn","sct_ab_gwth"]].fillna(9)
    df_c["sct_ab_invg"] = df_c["sct_ab_invg"].fillna(1)
    df_c.dropna(axis = 0, how="any", inplace= True )
    def update(row):
        if pd.isna(row["lesionsize"]):
            if row["de_stag"]<310:
                return 1
            elif row["de_stag"]>=310:
                return 25
            return row["lesionsize"]
        else:
            return row["lesionsize"]



    df_v["lesionsize"] = df_v.apply(update,axis = 1)
    df_v.dropna(axis = 0, how="any", inplace= True )

    return df_x, df_y, df_z, df_c, df_v


def merge(df_x, df_y, df_z, df_c, df_v):
    df_c = df_c.rename(columns={"sct_ab_code":"sct_ab_desc"})
    df = df_y.merge(df_x,on = "pid", how = "outer").merge(df_z, on =["pid","study_yr"],how = "outer").merge(df_c,on =["pid","study_yr","sct_ab_desc"],how = "outer").merge(df_v, on="pid", how = "outer")
    df = df.drop(columns=["sct_ab_num_y","study_yr_y"])
    df[["study_yr_x","sct_ab_desc","sct_ab_num_x","de_stag","lesionsize","lc_morph","lc_behav","lc_grade","lc_order"]] = df[["study_yr_x","sct_ab_desc","sct_ab_num_x","de_stag","lesionsize","lc_morph","lc_behav","lc_grade","lc_order"]].fillna(0)
    df[["sct_ab_preexist", "sct_ab_attn", "sct_ab_gwth", "sct_ab_invg"]] = df[["sct_ab_preexist", "sct_ab_attn", "sct_ab_gwth", "sct_ab_invg"]].fillna(1)
    df = df.dropna()
    
    return df


In [7]:
df_x, df_y, df_z, df_c, df_v = load("nlst_780_screen_idc_20210527.csv","nlst_780_prsn_idc_20210527.csv","nlst_780_ctab_idc_20210527.csv","nlst_780_ctabc_idc_20210527.csv","nlst_780_canc_idc_20210527.csv")
df_x, df_y, df_z, df_c, df_v=cleanup(df_x, df_y, df_z, df_c, df_v)
df_x, df_y, df_z, df_c, df_v=data(df_x, df_y, df_z, df_c, df_v)                                
df = merge(df_x, df_y, df_z, df_c, df_v)

In [8]:
# going to remove all the patients without cancer
df = df.loc[df["de_stag"] != 0]

In [11]:
# I have to turn the target column "de_stag" into integers. It is currently in type float
df["de_stag"] = df["de_stag"].astype(int)

In [13]:
# XGboost model requires the target value to be from [0-10] values instead of the current values that I have in the target column. 
# Will do a .map to change the values from the current version to the required ones. Will have to change the key again at the end or make a new one to know what each refers to. 
key = {110:0 , 120:1 , 210:2 , 220:3 , 310:4 , 320:5 , 400:6 , 888:7 , 900:8 , 994:9 , 999:10}
df["de_stag"] = df["de_stag"].map(key)

# XGboost model

In [16]:
X = df.drop("de_stag",axis = 1)
y = df["de_stag"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [18]:
xgbmodel = XGBClassifier()

In [20]:
xgbmodel.get_params()

{'objective': 'binary:logistic',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': None,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [22]:
# I will define specific values for the parameters to fit my goal
params={"objective": ["multi:softprob"],
        "n_estimators": [2000],
        "max_depth": [8,10,12],
        "learning_rate":[0.05,0.1],
        "subsample": [0.7],
        "colsample_bylevel": [0.8,1],
        "colsample_bynode": [0.8,1],
        "colsample_bytree": [0.8,1],
        "gamma": [0.2],
        "verbosity": [2],
        "min_child_weight": [1],
        "n_jobs": [6]}


In [24]:
n_cv = 2 # train test split
xgbmodel = XGBClassifier() #model that I will be using


# scoring: Techniques I will use to objectify the findings
scoring = ['roc_auc', 'accuracy']



# Create a gridsearch with the scoring, parameters and model.
grid_solver = GridSearchCV(estimator = xgbmodel, # model to train
                           param_grid = params, # param_grid
                           scoring = scoring,
                           cv = n_cv,
                           refit = 'accuracy',
                           verbose = 2)

In [26]:
grid_solver.fit(X_train,y_train, 
            eval_set=[(X_train, y_train), (X_test, y_test)])

Fitting 2 folds for each of 48 candidates, totalling 96 fits
[0]	validation_0-mlogloss:2.26166	validation_1-mlogloss:2.26391
[1]	validation_0-mlogloss:2.14976	validation_1-mlogloss:2.15324
[2]	validation_0-mlogloss:2.05377	validation_1-mlogloss:2.05860
[3]	validation_0-mlogloss:1.96577	validation_1-mlogloss:1.97259
[4]	validation_0-mlogloss:1.88343	validation_1-mlogloss:1.89209
[5]	validation_0-mlogloss:1.81133	validation_1-mlogloss:1.82178
[6]	validation_0-mlogloss:1.74592	validation_1-mlogloss:1.75837
[7]	validation_0-mlogloss:1.68396	validation_1-mlogloss:1.69765
[8]	validation_0-mlogloss:1.62871	validation_1-mlogloss:1.64392
[9]	validation_0-mlogloss:1.57539	validation_1-mlogloss:1.59202
[10]	validation_0-mlogloss:1.52526	validation_1-mlogloss:1.54364
[11]	validation_0-mlogloss:1.47715	validation_1-mlogloss:1.49719
[12]	validation_0-mlogloss:1.43092	validation_1-mlogloss:1.45274
[13]	validation_0-mlogloss:1.38948	validation_1-mlogloss:1.41205
[14]	validation_0-mlogloss:1.34834	vali

C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 137, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\metrics\_ranking.py", line 633, in roc_auc_score
    raise ValueError("multi_class must be in ('ovo', 'ovr')")
Va

[21]	validation_0-mlogloss:1.13014	validation_1-mlogloss:1.16189
[22]	validation_0-mlogloss:1.10169	validation_1-mlogloss:1.13439
[23]	validation_0-mlogloss:1.07706	validation_1-mlogloss:1.11031
[24]	validation_0-mlogloss:1.05395	validation_1-mlogloss:1.08823
[25]	validation_0-mlogloss:1.03003	validation_1-mlogloss:1.06553
[26]	validation_0-mlogloss:1.00745	validation_1-mlogloss:1.04391
[27]	validation_0-mlogloss:0.98637	validation_1-mlogloss:1.02398
[28]	validation_0-mlogloss:0.96641	validation_1-mlogloss:1.00478
[29]	validation_0-mlogloss:0.94608	validation_1-mlogloss:0.98496
[30]	validation_0-mlogloss:0.92694	validation_1-mlogloss:0.96717
[31]	validation_0-mlogloss:0.90686	validation_1-mlogloss:0.94798
[32]	validation_0-mlogloss:0.88798	validation_1-mlogloss:0.93001
[33]	validation_0-mlogloss:0.87084	validation_1-mlogloss:0.91370
[34]	validation_0-mlogloss:0.85276	validation_1-mlogloss:0.89604
[35]	validation_0-mlogloss:0.83731	validation_1-mlogloss:0.88143
[36]	validation_0-mloglos

C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 137, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\metrics\_ranking.py", line 633, in roc_auc_score
    raise ValueError("multi_class must be in ('ovo', 'ovr')")
Va

[19]	validation_0-mlogloss:1.10187	validation_1-mlogloss:1.14242
[20]	validation_0-mlogloss:1.06991	validation_1-mlogloss:1.11191
[21]	validation_0-mlogloss:1.04032	validation_1-mlogloss:1.08398
[22]	validation_0-mlogloss:1.01045	validation_1-mlogloss:1.05513
[23]	validation_0-mlogloss:0.98350	validation_1-mlogloss:1.02904
[24]	validation_0-mlogloss:0.95912	validation_1-mlogloss:1.00545
[25]	validation_0-mlogloss:0.93370	validation_1-mlogloss:0.98086
[26]	validation_0-mlogloss:0.91054	validation_1-mlogloss:0.95939
[27]	validation_0-mlogloss:0.88862	validation_1-mlogloss:0.93878
[28]	validation_0-mlogloss:0.86954	validation_1-mlogloss:0.92048
[29]	validation_0-mlogloss:0.84932	validation_1-mlogloss:0.90135
[30]	validation_0-mlogloss:0.82998	validation_1-mlogloss:0.88276
[31]	validation_0-mlogloss:0.81152	validation_1-mlogloss:0.86539
[32]	validation_0-mlogloss:0.79304	validation_1-mlogloss:0.84812
[33]	validation_0-mlogloss:0.77596	validation_1-mlogloss:0.83228
[34]	validation_0-mloglos

...

C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 137, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\metrics\_ranking.py", line 633, in roc_auc_score
    raise ValueError("multi_class must be in ('ovo', 'ovr')")
Va

[24]	validation_0-mlogloss:0.94253	validation_1-mlogloss:0.98607
[25]	validation_0-mlogloss:0.91895	validation_1-mlogloss:0.96422
[26]	validation_0-mlogloss:0.89575	validation_1-mlogloss:0.94181
[27]	validation_0-mlogloss:0.87310	validation_1-mlogloss:0.91989
[28]	validation_0-mlogloss:0.85185	validation_1-mlogloss:0.89991
[29]	validation_0-mlogloss:0.82960	validation_1-mlogloss:0.87854
[30]	validation_0-mlogloss:0.80937	validation_1-mlogloss:0.85920
[31]	validation_0-mlogloss:0.79102	validation_1-mlogloss:0.84137
[32]	validation_0-mlogloss:0.77191	validation_1-mlogloss:0.82321
[33]	validation_0-mlogloss:0.75448	validation_1-mlogloss:0.80667
[34]	validation_0-mlogloss:0.73813	validation_1-mlogloss:0.79103
[35]	validation_0-mlogloss:0.72205	validation_1-mlogloss:0.77596
[36]	validation_0-mlogloss:0.70619	validation_1-mlogloss:0.76106
[37]	validation_0-mlogloss:0.69103	validation_1-mlogloss:0.74660
[38]	validation_0-mlogloss:0.67583	validation_1-mlogloss:0.73199
[39]	validation_0-mloglos

C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 137, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\metrics\_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\utils\_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Gaspar\anaconda3\Lib\site-packages\sklearn\metrics\_ranking.py", line 633, in roc_auc_score
    raise ValueError("multi_class must be in ('ovo', 'ovr')")
Va

[23]	validation_0-mlogloss:0.61587	validation_1-mlogloss:0.67533
[24]	validation_0-mlogloss:0.59737	validation_1-mlogloss:0.65741
[25]	validation_0-mlogloss:0.57939	validation_1-mlogloss:0.64024
[26]	validation_0-mlogloss:0.56271	validation_1-mlogloss:0.62511
[27]	validation_0-mlogloss:0.54692	validation_1-mlogloss:0.61057
[28]	validation_0-mlogloss:0.53247	validation_1-mlogloss:0.59736
[29]	validation_0-mlogloss:0.51791	validation_1-mlogloss:0.58423
[30]	validation_0-mlogloss:0.50465	validation_1-mlogloss:0.57194
[31]	validation_0-mlogloss:0.49167	validation_1-mlogloss:0.56023
[32]	validation_0-mlogloss:0.47990	validation_1-mlogloss:0.54880
[33]	validation_0-mlogloss:0.46922	validation_1-mlogloss:0.53866
[34]	validation_0-mlogloss:0.45844	validation_1-mlogloss:0.52812
[35]	validation_0-mlogloss:0.44831	validation_1-mlogloss:0.51943
[36]	validation_0-mlogloss:0.43923	validation_1-mlogloss:0.51114
[37]	validation_0-mlogloss:0.43077	validation_1-mlogloss:0.50328
[38]	validation_0-mloglos

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [27]:
grid_solver.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
              device=None, early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.2, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.05, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=12,
              max_leaves=None, min_child_weight=1, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=2000,
              n_jobs=6, num_parallel_tree=None, objective='multi:softprob', ...)

In [ ]:
yhat=grid_solver.predict(X_train)
accuracy_score(y_train,yhat)

In [ ]:
yhat=grid_solver.predict(X_test)
accuracy_score(y_test,yhat)

In [ ]:
# By the results, we can conclude the program is overfitting as the train results are much higher than the test. 
# These results are better than the one of the previous tries I have completed

In [ ]:
# to get the probability of the model
probs = grid_solver.predict_proba(X_test)

In [ ]:
probs_df = pd.DataFrame(probs, columns=[f'prob_cancer_type_{i}' for i in range(probs.shape[1])])

In [ ]:
probs_df

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [ ]:
cm = confusion_matrix(y_test, yhat)

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=grid_solver.classes_)
disp.plot(cmap=plt.cm.Blues)

# Show the plot
plt.show()